[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# La especificación *JSONSchema*.

In [ ]:
from flask import Flask, jsonify, request, abort
from json import loads, dumps
from jsonschema import validate
from jsonschema.exceptions import ValidationError
from data import ruta, esquema_alumno 

## El *script* ```data/alumnos.py```.

El *script* ```data/alumnos.py``` define un objeto tipo  ```list``` que contiene los datos de alumnos ficticios.

In [ ]:
%pycat data/alumnos.py

## La función ```accede_base()```.

In [ ]:
def accede_base(ruta, cuenta):
    dato = None
    with open(ruta, 'tr') as f:
        base = eval(f.read())
    for item in base:
        if item['cuenta'] == cuenta:
            dato = item
    return base, dato

## Funciones de validación de datos.

### Función que valida las reglas de los datos.

* Validará que los datos tengan una estructura que con la descripción de ```esquema_alumno```.

In [ ]:
esquema_alumno

### Función que valida que el mensaje contiene todos los campos obligatorios.

In [ ]:
def recurso_completo(candidato):
    try:
        validate(candidato, esquema_alumno)
        if set(esquema_alumno['properties']).issuperset(set(candidato)):
            return True
        else:
            raise ValidationError('Invalid data')
    except ValidationError as e:
        abort(400, e)

## Código del servidor.

* El servidor correrá en [localhost:5000/api/](localhost:5000/api/). Si se accede a la raíz, se desplegará un listado de todos los alumnos en formato JSON.
* El servidor soporta los métodos:
    * **GET**: para obtener la información de un alumno por su número de cuenta.
    * **POST**: para crear un registro nuevo.
    * **PUT**: para sustituir por completo un registro existente.
    * **PATCH**: para modificar ciertos datos de un registro existente.
    * **DELETE**: para eliminar un registro existente.

In [ ]:
app = Flask(__name__)


@app.route('/api/', methods=['GET'])
def index():
    with open(ruta, 'tr') as base:    
        return jsonify(eval(base.read()))

    
@app.route('/api/<int:cuenta>', methods=['GET', 'POST', 'PUT', 'PATCH', 'DELETE'])
def api(cuenta):   
    if request.method == 'GET':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            return jsonify(alumno)
        else:
            abort(404)
    if request.method == 'DELETE':               
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            base.remove(alumno)
            with open(ruta, 'wt') as f:
                f.write(str(base))
            return jsonify(alumno)
        else:
            abort(404)       
    if request.method == 'POST':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            abort(409)
        else:
            candidato = loads(request.data)
            candidato['cuenta'] = cuenta
            if recurso_completo(candidato):
                base.append(candidato)
                with open(ruta, 'wt') as f:
                    f.write(str(base))
                return jsonify(candidato)            
    if request.method == 'PUT':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            candidato = loads(request.data)
            candidato['cuenta'] = cuenta
            if recurso_completo(candidato):
                base.remove(alumno)
                base.append(candidato)
                with open(ruta, 'wt') as f:
                    f.write(str(base))
                return jsonify(candidato)
        else:
            abort(404)
    if request.method == 'PATCH':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            alumno.update(loads(request.data))
            if recurso_completo(alumno):
                base.remove(alumno)
                base.append(alumno)
                with open(ruta, 'wt') as f:
                    f.write(str(base))
                return jsonify(alumno)
        else:
            abort(404)

In [ ]:
app.run('0.0.0.0')

### Notas:  
* **No reinicie o detenga el kernel de la notebook hasta que los clientes que accedan a esta aplicación hayan terminado sus sesiones.**
* Debido a que el código de la celda de arriba levanta el servidor de Flask, ésta se ejecutará indefinidamente y desplegará los mensajes de respuesta a las peticiones de los clientes que se conecten. 
* La notebook [14_cliente_api_rest.ipynb](12_cliente_api_rest.ipynb) contiene al cliente para este servidor.


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>